# Session 2: Custom Hooks & Best Practices
## Comprehensive Training Guide for Fresh Engineering Graduates

---

## Table of Contents
1. [Introduction to Custom Hooks](#introduction-to-custom-hooks)
2. [What Qualifies as a Custom Hook](#what-qualifies-as-a-custom-hook)
3. [Naming Conventions & Rules of Hooks](#naming-conventions--rules-of-hooks)
4. [Extracting Logic from Components](#extracting-logic-from-components)
5. [Sharing Stateful vs UI Logic](#sharing-stateful-vs-ui-logic)
6. [Parameterized & Composable Hooks](#parameterized--composable-hooks)
7. [Handling Side Effects in Hooks](#handling-side-effects-in-hooks)
8. [When NOT to Create a Custom Hook](#when-not-to-create-a-custom-hook)
9. [Testing Custom Hooks](#testing-custom-hooks)
10. [Real-World Patterns](#real-world-patterns)
11. [Exercises & Assessment](#exercises--assessment)

---

## Introduction to Custom Hooks

### What is a Custom Hook?

A **custom hook** is a JavaScript function that:
1. Has a name starting with `use` (e.g., `useCounter`, `useFetch`)
2. Calls other hooks (built-in or custom)
3. Returns state/functions/objects that components can use
4. Allows you to extract and reuse logic across components

### Why Custom Hooks Matter

Before custom hooks, code reuse in React was difficult. Common patterns for reuse:

| Pattern | Problem | Solution |
|---------|---------|----------|
| **Copy-paste logic** | ❌ Duplicate code everywhere, hard to maintain | ✅ Custom hook (single source of truth) |
| **Render Props** | ❌ Deeply nested, hard to read | ✅ Custom hook (clean function call) |
| **Higher-Order Components** | ❌ Wrapper hell, naming confusion | ✅ Custom hook (simple and direct) |

---

## What Qualifies as a Custom Hook

### The Golden Rule: Use Hooks Inside

A custom hook must call at least one built-in or custom hook:

```javascript
// ✅ Valid Custom Hook - calls useState
function useCounter(initialValue = 0) {
  const [count, setCount] = useState(initialValue);
  
  const increment = () => setCount(count + 1);
  const decrement = () => setCount(count - 1);
  
  return { count, increment, decrement };
}

// ✅ Valid Custom Hook - calls useEffect
function useFetch(url) {
  const [data, setData] = useState(null);
  
  useEffect(() => {
    fetch(url).then(r => r.json()).then(setData);
  }, [url]);
  
  return data;
}

// ✅ Valid Custom Hook - calls other custom hooks
function useUser(userId) {
  const user = useFetch(`/api/users/${userId}`);
  const posts = useFetch(`/api/users/${userId}/posts`);
  
  return { user, posts };
}

// ❌ NOT a Custom Hook - doesn't call any hooks
function calculateSum(a, b) {
  return a + b;
}
// This is just a utility function, not a custom hook!
```

### Hook vs Utility Function

**Know the difference:**

```javascript
// ✅ CUSTOM HOOK - calls React hooks
function useAsync(asyncFunction, immediate = true) {
  const [status, setStatus] = useState('idle');
  const [data, setData] = useState(null);
  const [error, setError] = useState(null);

  useEffect(() => {
    if (!immediate) return;
    
    asyncFunction()
      .then(data => setData(data))
      .catch(err => setError(err));
  }, [asyncFunction, immediate]);

  return { status, data, error };
}

// ✅ UTILITY FUNCTION - no React hooks
function formatDate(date) {
  return new Date(date).toLocaleDateString();
}

function calculateTotal(items) {
  return items.reduce((sum, item) => sum + item.price, 0);
}
```

---

## Naming Conventions & Rules of Hooks

### Rule 1: Always Start with "use"

This naming convention is **critical** because React uses it to enforce the Rules of Hooks.

```javascript
// ✅ Correct naming
function useWindowWidth() { ... }
function useLocalStorage(key) { ... }
function useAsync(fn) { ... }
function useFormInput(initialValue) { ... }

// ❌ Wrong naming (violates convention)
function getWindowWidth() { ... }      // Doesn't start with 'use'
function fetchData(url) { ... }         // Doesn't start with 'use'
function createCounter() { ... }       // Starts with 'create', not 'use'
```

### Rule 2: Only Call at Top Level

Hooks must be called at the **top level** of your function component or custom hook.

```javascript
// ❌ WRONG - Hook inside a condition
function BadComponent({ shouldFetch }) {
  if (shouldFetch) {
    const data = useFetch('/api/data'); // ❌ NOT at top level!
  }
}

// ✅ CORRECT - Condition inside the effect
function GoodComponent({ shouldFetch }) {
  const data = useFetch('/api/data'); // Top level!
  
  useEffect(() => {
    if (!shouldFetch) return; // Condition goes INSIDE
  }, [shouldFetch]);
}

// ❌ WRONG - Hook inside a loop
function BadComponent({ ids }) {
  for (let id of ids) {
    const user = useUser(id); // ❌ NOT at top level!
  }
}

// ✅ CORRECT - Use effect with array inside
function GoodComponent({ ids }) {
  const [users, setUsers] = useState(null);
  
  useEffect(() => {
    Promise.all(ids.map(id => fetch(`/api/users/${id}`)))
      .then(responses => Promise.all(responses.map(r => r.json())))
      .then(setUsers);
  }, [ids]);
}

// ❌ WRONG - Hook inside a function
function BadComponent() {
  const handleClick = () => {
    const [count, setCount] = useState(0); // ❌ NOT at top level!
  };
}

// ✅ CORRECT - Hook at top level
function GoodComponent() {
  const [count, setCount] = useState(0); // Top level!
  
  const handleClick = () => {
    setCount(count + 1); // Use the state in handler
  };
}
```

### Rule 3: Only Call from React Functions

Hooks can only be called from:
1. React component functions
2. Other custom hooks

```javascript
// ✅ Calling from a React component
function MyComponent() {
  const [count, setCount] = useState(0);
}

// ✅ Calling from a custom hook
function useCounter() {
  const [count, setCount] = useState(0);
  return { count, setCount };
}

// ❌ Calling from a regular function
const handleClick = () => {
  const [count, setCount] = useState(0); // ❌ NOT in a React context!
};

// ❌ Calling from a class method
class MyClass {
  render() {
    const [count, setCount] = useState(0); // ❌ Can't use hooks in classes!
  }
}
```

---

## Extracting Logic from Components

### Step-by-Step Extraction Process

**Original Component with Duplicate Logic:**

```javascript
// Two components with the same logic - violates DRY principle
function UserProfile() {
  const [user, setUser] = useState(null);
  const [loading, setLoading] = useState(true);
  const [error, setError] = useState(null);

  useEffect(() => {
    setLoading(true);
    fetch(`/api/user/me`)
      .then(r => {
        if (!r.ok) throw new Error('Failed to fetch');
        return r.json();
      })
      .then(data => {
        setUser(data);
        setLoading(false);
      })
      .catch(err => {
        setError(err.message);
        setLoading(false);
      });
  }, []);

  return (
    <>
      {loading && <p>Loading...</p>}
      {error && <p>Error: {error}</p>}
      {user && <h1>{user.name}</h1>}
    </>
  );
}

function PostsList() {
  const [posts, setPosts] = useState(null);
  const [loading, setLoading] = useState(true);
  const [error, setError] = useState(null);

  useEffect(() => {
    setLoading(true);
    fetch(`/api/posts`)
      .then(r => {
        if (!r.ok) throw new Error('Failed to fetch');
        return r.json();
      })
      .then(data => {
        setPosts(data);
        setLoading(false);
      })
      .catch(err => {
        setError(err.message);
        setLoading(false);
      });
  }, []);

  return (
    <>
      {loading && <p>Loading...</p>}
      {error && <p>Error: {error}</p>}
      {posts && posts.map(post => <div key={post.id}>{post.title}</div>)}
    </>
  );
}
```

**Step 1: Identify Common Pattern**
Both components fetch data, handle loading/error, and return values.

**Step 2: Extract into Custom Hook**

```javascript
// New custom hook - single source of truth
function useFetch(url) {
  const [data, setData] = useState(null);
  const [loading, setLoading] = useState(true);
  const [error, setError] = useState(null);

  useEffect(() => {
    setLoading(true);
    setError(null);
    
    fetch(url)
      .then(r => {
        if (!r.ok) throw new Error('Failed to fetch');
        return r.json();
      })
      .then(data => {
        setData(data);
        setLoading(false);
      })
      .catch(err => {
        setError(err.message);
        setLoading(false);
      });
  }, [url]);

  return { data, loading, error };
}
```

**Step 3: Use Hook in Components**

```javascript
// Clean, readable components
function UserProfile() {
  const { data: user, loading, error } = useFetch('/api/user/me');

  return (
    <>
      {loading && <p>Loading...</p>}
      {error && <p>Error: {error}</p>}
      {user && <h1>{user.name}</h1>}
    </>
  );
}

function PostsList() {
  const { data: posts, loading, error } = useFetch('/api/posts');

  return (
    <>
      {loading && <p>Loading...</p>}
      {error && <p>Error: {error}</p>}
      {posts && posts.map(post => <div key={post.id}>{post.title}</div>)}
    </>
  );
}
```

**Benefits:**
- ✅ Code is DRY (Don't Repeat Yourself)
- ✅ Single source of truth (easier to maintain and test)
- ✅ Consistent behavior across components
- ✅ Easy to extend with new features

---

## Sharing Stateful vs UI Logic

### Key Distinction

**Stateful Logic** = State, effects, and business logic (can be extracted to hooks)
**UI Logic** = How things look and render (stays in components)

```javascript
// ❌ WRONG - Mixing state and UI in a hook
function useUserProfile() {
  const [user, setUser] = useState(null);

  // ❌ Hook shouldn't return JSX/UI
  return (
    <div>
      <h1>{user?.name}</h1>
      <p>{user?.email}</p>
    </div>
  );
}

// ✅ CORRECT - Hook returns state only
function useUserProfile(userId) {
  const [user, setUser] = useState(null);
  const [loading, setLoading] = useState(true);

  useEffect(() => {
    fetch(`/api/users/${userId}`)
      .then(r => r.json())
      .then(data => {
        setUser(data);
        setLoading(false);
      });
  }, [userId]);

  return { user, loading };
}

// Component handles the UI
function UserProfileCard({ userId }) {
  const { user, loading } = useUserProfile(userId);

  if (loading) return <p>Loading...</p>;
  if (!user) return null;

  return (
    <div className="card">
      <h1>{user.name}</h1>
      <p>{user.email}</p>
    </div>
  );
}
```

### Example: Form Logic Extraction

```javascript
// ✅ Extract form validation logic (stateful)
function useForm(initialValues, onSubmit) {
  const [values, setValues] = useState(initialValues);
  const [errors, setErrors] = useState({});
  const [touched, setTouched] = useState({});
  const [isSubmitting, setIsSubmitting] = useState(false);

  const handleChange = (e) => {
    const { name, value } = e.target;
    setValues(prev => ({
      ...prev,
      [name]: value
    }));
  };

  const handleBlur = (e) => {
    const { name } = e.target;
    setTouched(prev => ({
      ...prev,
      [name]: true
    }));
  };

  const handleSubmit = async (e) => {
    e.preventDefault();
    setIsSubmitting(true);
    
    try {
      await onSubmit(values);
    } catch (err) {
      setErrors(err.fieldErrors || {});
    } finally {
      setIsSubmitting(false);
    }
  };

  return {
    values,
    errors,
    touched,
    isSubmitting,
    handleChange,
    handleBlur,
    handleSubmit
  };
}

// ✅ Keep UI logic in component
function LoginForm() {
  const form = useForm(
    { email: '', password: '' },
    async (values) => {
      const response = await fetch('/api/login', {
        method: 'POST',
        body: JSON.stringify(values)
      });
      if (!response.ok) throw new Error('Login failed');
      return response.json();
    }
  );

  return (
    <form onSubmit={form.handleSubmit}>
      <div>
        <input
          name="email"
          value={form.values.email}
          onChange={form.handleChange}
          onBlur={form.handleBlur}
        />
        {form.touched.email && form.errors.email && (
          <span className="error">{form.errors.email}</span>
        )}
      </div>

      <button type="submit" disabled={form.isSubmitting}>
        {form.isSubmitting ? 'Logging in...' : 'Login'}
      </button>
    </form>
  );
}
```

---

## Parameterized & Composable Hooks

### Parameterized Hooks (Accept Arguments)

```javascript
// Hook that accepts parameters
function useCounter(initialValue = 0, step = 1) {
  const [count, setCount] = useState(initialValue);

  const increment = () => setCount(count + step);
  const decrement = () => setCount(count - step);
  const reset = () => setCount(initialValue);

  return { count, increment, decrement, reset };
}

// Usage with different parameters
function CounterComponent1() {
  const counter = useCounter(); // Uses defaults: 0, 1
  return <div>Count: {counter.count}</div>;
}

function CounterComponent2() {
  const counter = useCounter(10, 5); // Start at 10, step of 5
  return <div>Count: {counter.count}</div>;
}
```

### Hook Composition (Hooks Calling Other Hooks)

```javascript
// Base hook
function useFetch(url) {
  const [data, setData] = useState(null);
  const [loading, setLoading] = useState(true);
  const [error, setError] = useState(null);

  useEffect(() => {
    fetch(url)
      .then(r => r.json())
      .then(setData)
      .catch(setError)
      .finally(() => setLoading(false));
  }, [url]);

  return { data, loading, error };
}

// Composed hook - builds on useFetch
function useUserData(userId) {
  const user = useFetch(`/api/users/${userId}`);
  
  if (user.data) {
    // Fetch additional data based on first result
    const posts = useFetch(`/api/users/${userId}/posts`);
    const followers = useFetch(`/api/users/${userId}/followers`);
    
    return {
      user: user.data,
      posts: posts.data,
      followers: followers.data,
      loading: user.loading || posts.loading || followers.loading,
      error: user.error || posts.error || followers.error
    };
  }

  return {
    user: null,
    posts: null,
    followers: null,
    loading: user.loading,
    error: user.error
  };
}

// Even higher-level composition
function useUserProfile(userId) {
  const userData = useUserData(userId);
  const [profile, setProfile] = useState(null);

  useEffect(() => {
    if (userData.user && userData.posts) {
      setProfile({
        ...userData.user,
        postCount: userData.posts.length,
        followerCount: userData.followers?.length || 0
      });
    }
  }, [userData.user, userData.posts, userData.followers]);

  return profile;
}
```

---

## Handling Side Effects in Hooks

### Managing Subscriptions

```javascript
function useSubscription(channel) {
  const [data, setData] = useState(null);

  useEffect(() => {
    // Subscribe
    const subscription = EventBus.subscribe(channel, setData);

    // Cleanup: Unsubscribe when component unmounts or channel changes
    return () => {
      subscription.unsubscribe();
    };
  }, [channel]);

  return data;
}
```

### Managing Timers & Intervals

```javascript
function useInterval(callback, delay) {
  const savedCallback = useRef(callback);

  // Remember the latest callback if it changes
  useEffect(() => {
    savedCallback.current = callback;
  }, [callback]);

  useEffect(() => {
    if (delay === null) return; // Allow pausing

    const interval = setInterval(() => {
      savedCallback.current();
    }, delay);

    return () => clearInterval(interval); // Cleanup
  }, [delay]);
}

// Usage
function Timer() {
  const [seconds, setSeconds] = useState(0);

  useInterval(() => {
    setSeconds(s => s + 1);
  }, 1000);

  return <p>Elapsed: {seconds}s</p>;
}
```

### Managing WebSocket Connections

```javascript
function useWebSocket(url) {
  const [data, setData] = useState(null);
  const [error, setError] = useState(null);
  const [status, setStatus] = useState('connecting');

  useEffect(() => {
    const ws = new WebSocket(url);

    ws.onopen = () => setStatus('connected');
    ws.onmessage = (event) => {
      try {
        setData(JSON.parse(event.data));
      } catch (err) {
        setError('Invalid message format');
      }
    };
    ws.onerror = () => setStatus('error');
    ws.onclose = () => setStatus('disconnected');

    // Cleanup: Close connection on unmount
    return () => {
      if (ws.readyState === WebSocket.OPEN) {
        ws.close();
      }
    };
  }, [url]);

  return { data, error, status };
}
```

---

## When NOT to Create a Custom Hook

### Anti-Pattern 1: Hook for Simple Calculations

```javascript
// ❌ WRONG - Don't create a hook for simple logic
function useSum(a, b) {
  return a + b;
}

// ✅ CORRECT - Just use a regular function
function sum(a, b) {
  return a + b;
}

// Usage
function Calculator() {
  const result = sum(5, 10); // Much clearer!
}
```

### Anti-Pattern 2: Hook Wrapping a Single Built-in Hook

```javascript
// ❌ WRONG - Unnecessary wrapper
function useCustomState(initialValue) {
  return useState(initialValue);
}

// ✅ CORRECT - Just use useState directly
function MyComponent() {
  const [value, setValue] = useState(0);
}
```

### Anti-Pattern 3: Hook for Global State (Use State Management)

```javascript
// ❌ WRONG - Using hooks for global app state
function useGlobalUser() {
  const [user, setUser] = useState(null);
  // This doesn't persist across components!
}

// ✅ CORRECT - Use Context or state management library
const UserContext = React.createContext();

function App() {
  const [user, setUser] = useState(null);
  
  return (
    <UserContext.Provider value={user}>
      {/* All children can access user */}
    </UserContext.Provider>
  );
}

function MyComponent() {
  const user = useContext(UserContext);
}
```

### Anti-Pattern 4: Hook Returning JSX

```javascript
// ❌ WRONG - Hook shouldn't return JSX
function useUserUI(user) {
  return (
    <div>
      <h1>{user.name}</h1>
      <p>{user.email}</p>
    </div>
  );
}

// ✅ CORRECT - Hook returns data, component renders
function useUser(userId) {
  const [user, setUser] = useState(null);
  // ... fetch logic ...
  return user;
}

function UserDisplay({ userId }) {
  const user = useUser(userId);
  
  if (!user) return null;
  
  return (
    <div>
      <h1>{user.name}</h1>
      <p>{user.email}</p>
    </div>
  );
}
```

---

## Testing Custom Hooks

### Testing with React Testing Library

```javascript
import { renderHook, act, waitFor } from '@testing-library/react';
import { useCounter } from './useCounter';

describe('useCounter', () => {
  it('should initialize with default value', () => {
    const { result } = renderHook(() => useCounter());
    
    expect(result.current.count).toBe(0);
  });

  it('should initialize with custom value', () => {
    const { result } = renderHook(() => useCounter(10));
    
    expect(result.current.count).toBe(10);
  });

  it('should increment count', () => {
    const { result } = renderHook(() => useCounter());
    
    act(() => {
      result.current.increment();
    });
    
    expect(result.current.count).toBe(1);
  });

  it('should decrement count', () => {
    const { result } = renderHook(() => useCounter(5));
    
    act(() => {
      result.current.decrement();
    });
    
    expect(result.current.count).toBe(4);
  });

  it('should reset count', () => {
    const { result } = renderHook(() => useCounter(5));
    
    act(() => {
      result.current.increment();
      result.current.increment();
      result.current.reset();
    });
    
    expect(result.current.count).toBe(5);
  });
});
```

### Testing Async Hooks

```javascript
import { renderHook, waitFor } from '@testing-library/react';
import { useFetch } from './useFetch';

describe('useFetch', () => {
  it('should fetch data', async () => {
    global.fetch = jest.fn(() =>
      Promise.resolve({
        ok: true,
        json: () => Promise.resolve({ id: 1, name: 'John' })
      })
    );

    const { result } = renderHook(() => useFetch('/api/user'));

    expect(result.current.loading).toBe(true);

    await waitFor(() => {
      expect(result.current.loading).toBe(false);
    });

    expect(result.current.data).toEqual({ id: 1, name: 'John' });
    expect(result.current.error).toBe(null);
  });

  it('should handle errors', async () => {
    global.fetch = jest.fn(() =>
      Promise.reject(new Error('Network error'))
    );

    const { result } = renderHook(() => useFetch('/api/user'));

    await waitFor(() => {
      expect(result.current.loading).toBe(false);
    });

    expect(result.current.error).toEqual(new Error('Network error'));
  });
});
```

---

## Real-World Patterns

### Pattern 1: useAsync (Advanced)

```javascript
function useAsync(asyncFunction, immediate = true) {
  const [status, setStatus] = useState('idle');
  const [data, setData] = useState(null);
  const [error, setError] = useState(null);
  const execute = useCallback(async () => {
    setStatus('pending');
    setData(null);
    setError(null);
    
    try {
      const response = await asyncFunction();
      setData(response);
      setStatus('success');
      return response;
    } catch (err) {
      setError(err);
      setStatus('error');
      throw err;
    }
  }, [asyncFunction]);

  useEffect(() => {
    if (immediate) {
      execute();
    }
  }, [execute, immediate]);

  return { execute, status, data, error };
}

// Usage
function DataComponent() {
  const { data, status, error, execute } = useAsync(
    () => fetch('/api/data').then(r => r.json()),
    true // Fetch immediately on mount
  );

  return (
    <div>
      {status === 'pending' && <p>Loading...</p>}
      {status === 'error' && <p>Error: {error.message}</p>}
      {status === 'success' && <p>Data: {JSON.stringify(data)}</p>}
      <button onClick={execute}>Refetch</button>
    </div>
  );
}
```

### Pattern 2: usePrevious (Track Previous Value)

```javascript
function usePrevious(value) {
  const ref = useRef();

  useEffect(() => {
    ref.current = value;
  }, [value]);

  return ref.current;
}

// Usage
function Counter() {
  const [count, setCount] = useState(0);
  const prevCount = usePrevious(count);

  return (
    <div>
      <p>Now: {count}, Before: {prevCount}</p>
      <button onClick={() => setCount(count + 1)}>Increment</button>
    </div>
  );
}
```

### Pattern 3: useMount & useUnmount

```javascript
function useMount(callback) {
  useEffect(() => {
    callback();
  }, []);
}

function useUnmount(callback) {
  useEffect(() => {
    return callback;
  }, []);
}

// Usage
function MyComponent() {
  useMount(() => {
    console.log('Component mounted');
    analytics.track('page_view');
  });

  useUnmount(() => {
    console.log('Component unmounting');
    analytics.track('page_leave');
  });

  return <div>...</div>;
}
```

---

## Exercises & Assessment

### Exercise 1: useLocalStorage Hook (Difficulty: Medium)
Create a hook that:
- Syncs state with localStorage
- Persists data across page reloads
- Handles JSON serialization/deserialization

```javascript
// Usage should be:
function Settings() {
  const [theme, setTheme] = useLocalStorage('theme', 'light');
}
```

### Exercise 2: useDebounce Hook (Difficulty: Medium)
Create a hook that debounces a value:
- Waits X milliseconds before returning new value
- Useful for search inputs

```javascript
// Usage:
function Search() {
  const [query, setQuery] = useState('');
  const debouncedQuery = useDebounce(query, 500);
}
```

### Exercise 3: usePagination Hook (Difficulty: Hard)
Create a hook that manages pagination logic:
- Accepts array of items and items per page
- Returns current page items and pagination controls
- Handles next/previous/goToPage

### Exercise 4: useMediaQuery Hook (Difficulty: Hard)
Create a hook that:
- Listens to CSS media queries
- Returns true/false if media query matches
- Useful for responsive design

```javascript
// Usage:
function ResponsiveComponent() {
  const isMobile = useMediaQuery('(max-width: 768px)');
  
  return isMobile ? <MobileView /> : <DesktopView />;
}
```

### Exercise 5: useRequest Hook (Difficulty: Hard)
Create a hook that:
- Handles GET/POST/PUT/DELETE requests
- Manages loading/error/data states
- Supports request cancellation
- Handles retries on failure

---

## Key Takeaways

✅ **Extract logic into hooks** when you find yourself repeating patterns
✅ **Separate concerns** - hooks for logic, components for UI
✅ **Use composition** - build complex hooks from simpler ones
✅ **Always clean up** - unsubscribe, cancel requests, clear timers
✅ **Test your hooks** - use renderHook from testing library
✅ **Follow naming conventions** - always start with "use"
✅ **Don't create hooks needlessly** - regular functions are fine for simple logic

---

*Last Updated: January 2026 | React 18+ | For Production Use*
